In [1]:
from PIL import Image
import torch
from tqdm.auto import tqdm
import numpy as np
from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base1B' # use base300M or base1B for better results
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

In [ ]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 3.0],
)

In [ ]:
# Load an image to condition on.
img = Image.open('example_data/first-dome.jpg')

# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
    samples = x

In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]
fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

### Load model for mesh

In [ ]:
print('creating SDF model...')
sdf_model = model_from_config(MODEL_CONFIGS['sdf'], device)
sdf_model.eval()

print('loading SDF model...')
sdf_model.load_state_dict(load_checkpoint('sdf', device))

In [ ]:

from point_e.util.pc_to_mesh import marching_cubes_mesh

pointcloud = sampler.output_to_point_clouds(samples)[0]
mesh = marching_cubes_mesh(
    pc=pointcloud,
    model=sdf_model,
    batch_size=4096,
    grid_size=32, # increase to 128 for resolution used in evals
    progress=True,
)
fig = plot_point_cloud(pointcloud, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))


### Save OBJ from mesh

In [ ]:
f = open("first-dome.obj", "w")
f.write("# OBJ file\n")
for i in range(len(mesh.verts)):
    f.write("v " + str(mesh.verts[i][0]) + " " + str(mesh.verts[i][1]) + " " + str(mesh.verts[i][2]) + " ")
    f.write("\n")

for i in range(len(mesh.normals)):
    f.write("vn " + str(mesh.normals[i][0]) + " " + str(mesh.normals[i][1]) + " " + str(mesh.normals[i][2]) + " ")
    f.write("\n")

for i in range(len(mesh.faces)):
    f.write("f " + str(mesh.faces[i][0]) + " " + str(mesh.faces[i][1]) + " " + str(mesh.faces[i][2]) + " ")
    f.write("\n")
f.close()